In [7]:
#| default_exp data_preparation

In [41]:
#| export
from pathlib import Path
from fastcore.all import *
import cv2
import numpy as np
from typing import List, Tuple, Union, Callable, Optional, Dict
from tqdm.auto import tqdm
import matplotlib.pyplot as plt 
import json

# first find the data

code highly taken from [here](https://github.com/bnsreenu/python_for_microscopists/blob/master/332%20-%20All%20about%20image%20annotations%E2%80%8B/binary_to_coco_V3.0.py)

In [40]:
trn_msk_path = Path(r'/home/hasan/Schreibtisch/projects/data/microscopy/patch_train_masks')
trn_img_path = Path(r'/home/hasan/Schreibtisch/projects/data/microscopy/patch_train_images/')

val_msk_path = Path(r'/home/hasan/Schreibtisch/projects/data/microscopy/patch_train_masks')
val_img_path = Path(r'/home/hasan/Schreibtisch/projects/data/microscopy/patch_train_images/')

trn_json_path = Path(r'/home/hasan/Schreibtisch/projects/data/microscopy/patch_mask_train_coco_format.json')
val_json_path = Path(r'/home/hasan/Schreibtisch/projects/data/microscopy/patch_mask_val_coco_format.json')
trn_msk_path.ls(), trn_img_path.ls()


((#1642) [Path('/home/hasan/Schreibtisch/projects/data/microscopy/patch_train_masks/img_162_p_9.png'),Path('/home/hasan/Schreibtisch/projects/data/microscopy/patch_train_masks/img_11_p_9.png'),Path('/home/hasan/Schreibtisch/projects/data/microscopy/patch_train_masks/img_70_p_5.png'),Path('/home/hasan/Schreibtisch/projects/data/microscopy/patch_train_masks/img_67_p_1.png'),Path('/home/hasan/Schreibtisch/projects/data/microscopy/patch_train_masks/img_74_p_9.png'),Path('/home/hasan/Schreibtisch/projects/data/microscopy/patch_train_masks/img_16_p_5.png'),Path('/home/hasan/Schreibtisch/projects/data/microscopy/patch_train_masks/img_23_p_1.png'),Path('/home/hasan/Schreibtisch/projects/data/microscopy/patch_train_masks/img_112_p_6.png'),Path('/home/hasan/Schreibtisch/projects/data/microscopy/patch_train_masks/img_121_p_11.png'),Path('/home/hasan/Schreibtisch/projects/data/microscopy/patch_train_masks/img_152_p_1.png')...],
 (#1642) [Path('/home/hasan/Schreibtisch/projects/data/microscopy/patc

# We need to create a format which can yolov8 works

 ### 1. First convert coco format

In [11]:
#| export
def get_contours(img:np.ndarray):
    'get contours from masks'

    _, thresh = cv2.threshold(img, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    return cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)[0]


In [30]:
#| export
def from_contr_to_annotation(
                            sn_cntr:list, # single contour
                            consider_min_area:bool=True,# whether to use min_area parameter
                            min_area:int=0,
                            )->Tuple:
    'Create annotation dict from  a single contour'
    bbox = cv2.boundingRect(sn_cntr)
    area = cv2.contourArea(sn_cntr)
    segmentation = sn_cntr.flatten().tolist()
    if consider_min_area:
        if area > min_area:
            return bbox, area, segmentation
        return None, None, None
    else:
        return bbox, area, segmentation



In [42]:
#| export
def get_mask_info(
        msk_path, 
        min_area=0,
        )->Tuple:
    all_masks = msk_path.ls()

    image_infos = []
    annotations = []
    annotation_id=0
    for idx, msk_fn in tqdm(enumerate(all_masks),total=len(all_masks)):
        image_id = idx +1
        file_name = msk_fn.name

        mask = cv2.imread(str(msk_fn), cv2.IMREAD_GRAYSCALE)
        height, width = mask.shape

        if file_name not in map(itemgetter('file_name'), image_infos):  
            image_info = {
                'id': image_id,
                'width': width,
                'height': height,
                'file_name': file_name}
            image_infos.append(image_info)
        else:
            image_info = list(filter(lambda x: x['file_name'] == file_name, image_infos))[0]
        
        cntrs = get_contours(mask)
        for cntr in cntrs:
            bbox, area, segmentation = from_contr_to_annotation(cntr, min_area=min_area)
            if bbox:
                annotation = {
                    'image_id': image_id,
                    'id': annotation_id,
                    'category_id': 1,
                    'iscrowd': 0,
                    'area': area,
                    'bbox': bbox,
                    'segmentation': [segmentation]
                }
                annotations.append(annotation)
                annotation_id +=1
    return image_infos, annotations, annotation_id


        

        



In [43]:
image_infos, annotations, annotation_id=get_mask_info(trn_msk_path, min_area=0)

  0%|          | 0/1642 [00:00<?, ?it/s]

In [33]:
category_ids = {
    "object": 1,
}

In [38]:
#| export
def process_masks(
        mask_path:Union[str, Path],
        json_path:Union[str, Path],
        category_ids:Dict,
        ):
    coco_format = {
        "info": {},
        "licenses": [],
        "images":[],
        "categories": [{"id":v, "name":k, "supercategory":k } for k, v in category_ids.items()],
        "annotations":[]

    }

    coco_format['images'], coco_format['annotations'], ann_cnt = get_mask_info(mask_path)
    with open(json_path, 'w') as f:
        json.dump(coco_format, f, sort_keys=True, indent=4)

#### Creating coco format json file for training set

In [44]:
process_masks(
    mask_path=trn_msk_path, 
    json_path=trn_json_path, 
    category_ids=category_ids)

  0%|          | 0/1642 [00:00<?, ?it/s]

#### Creating coco format for validation data

In [ ]:
process_masks(
    mask_path=val_msk_path, 
    json_path=val_json_path, 
    category_ids=category_ids)

  0%|          | 0/1642 [00:00<?, ?it/s]

### 2. Now convet to yolo format

In [47]:
#| export
def read_json(file_path):
    with open(file_path, 'r') as f:
        data = json.load(f)
    return data


In [32]:
#| hide
import nbdev; nbdev.nbdev_export()